In [1]:
## Import packages ##

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline


In [34]:
## Compute the camera calibration matrix and distortion coefficients given a set of chessboard images. ##

def calibrate_camera(cal_img_folder):
    #number of inside corners in x
    nx = 9 
    #number of inside corners in y
    ny = 6 
    #3D points in real world space
    objpoints = []
    #2D points in image plane
    imgpoints =[] 

    #Prepare object points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

    #Loop through calibration images
    cal_files = os.listdir(cal_img_folder)
    for file in cal_files:
        #Import calibration images
        cal_img = mpimg.imread(cal_img_folder + file) 
        #Convert to gray
        gray_cal_img = cv2.cvtColor(cal_img, cv2.COLOR_BGR2GRAY) 
        #Find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray_cal_img, (nx,ny), None) 
        #If corners are found, add object points and image points
        if ret == True: 
            imgpoints.append(corners)
            objpoints.append(objp)

    #Calulate matrix and distortion coefficients
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_cal_img.shape[::-1], None, None)
    
    return ret, mtx, dist, rvecs, tvecs


#Writeup/Test
#ret, mtx, dist, rvecs, tvecs = calibrate_camera("camera_cal/")

In [35]:
## Apply a distortion correction to raw images. ##

def undistort_img(raw_img, mtx, dist):
        #Unistort raw image
        undistort_img = cv2.undistort(raw_img, mtx, dist, None, mtx)
        #Fix coloring and save to folder
        r,g,b = cv2.split(undistort_img)
        undistorted_img = cv2.merge((b,g,r))
        return undistorted_img
        
        
#TEST
#folder = "test_images/"
#file = "test2.jpg"
#raw_files = os.listdir(folder)
#raw_img = mpimg.imread(folder + file)
#undistorted_img = undistort_img(raw_img, mtx, dist)
#gray_undistorted_img = cv2.cvtColor(undistorted_img, cv2.COLOR_RGB2GRAY)
#plt.imshow(gray_undistorted_img, cmap = 'gray')

#Writeup
#raw_img = mpimg.imread("test_images/test4.jpg")
#undistorted_img = undistort_img(raw_img, mtx, dist)
#cv2.imwrite("writeup_images/undistorted.jpg", undistorted_img)
#gray_undistorted_img = cv2.cvtColor(undistorted_img, cv2.COLOR_RGB2GRAY)

In [36]:
## Use color transforms, gradients, etc., to create a thresholded binary image. ##

#Absolute value of Sobel with inclusive threshold bounds
def abs_sobel_thresh(gray_undistorted_img, orient, sobel_kernel, thresh):
    #Determine orientation and find absolute value of sobel
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 1, 0, sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 0, 1, sobel_kernel))
    #Scale to 8-bit
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    #Create binary mask where sobel thresholds are met
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    #Return mask
    return binary_output

#Magnitude of the gradient with inclusive threshold bounds
def mag_thresh(gray_undistorted_img, sobel_kernel, thresh):
    #Take gradient in x and y
    sobelx = cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 1, 0, sobel_kernel)
    sobely = cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 0, 1, sobel_kernel)
    #Calculate magnitude
    sobel_mag = np.sqrt(sobelx**2 + sobely**2)
    #Scale to 8-bit (0-255) and convert to type = np.uint8
    scale_factor = np.max(sobel_mag)/255
    sobel_mag = (sobel_mag/scale_factor).astype(np.uint8)
    #Create binary mask where mag thresholds are met
    binary_output = np.zeros_like(sobel_mag)
    binary_output[(sobel_mag >= thresh[0]) & (sobel_mag <= thresh[1])] = 1
    #Return mask
    return binary_output

#Direction of the gradient with inclusive threshold bounds
def dir_thresh(gray_undistorted_img, sobel_kernel, thresh):
    #Take gradient in x and y
    sobelx = cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 1, 0, sobel_kernel)
    sobely = cv2.Sobel(gray_undistorted_img, cv2.CV_64F, 0, 1, sobel_kernel)
    #Take absolute values
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    #Calculate direction
    direction = np.arctan2(abs_sobely, abs_sobelx)
    #Create binary mask where direction thresholds are met
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    #Return mask
    return binary_output
     
#HLS color threshold with inclusive lower and exlusive upper bounds
def s_color_thresh(img, thresh):
    #Convert to HLS and seprate S filter
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    #Create binary mask where color thresholds are met
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary_output

#Stack all thresholds
def stack_thresholds(undistorted_gray_img, undistorted_img, ksize):
    gradx = abs_sobel_thresh(undistorted_gray_img, 'x', ksize, (30, 60))
    grady = abs_sobel_thresh(undistorted_gray_img, 'y', ksize, (30, 60))
    mag_binary = mag_thresh(undistorted_gray_img, ksize, (90, 180))
    dir_binary = dir_thresh(undistorted_gray_img, ksize, (0.7, 1.3))
    color_channel = s_color_thresh(undistorted_img, (90, 255))
    stack_all = np.zeros_like(dir_binary)
    stack_all[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (color_channel ==1)] = 1
    return stack_all


#TEST
#gradx = abs_sobel_thresh(gray_undistorted_img, 'x', ksize, (30, 60))
#grady = abs_sobel_thresh(gray_undistorted_img, 'y', ksize, (30, 60))
#mag_binary = mag_thresh(gray_undistorted_img, ksize, (90, 150))
#dir_binary = dir_thresh(gray_undistorted_img, ksize, (0.7, 1.3))
#color_channel = s_color_thresh(undistorted_img, (90, 255))
#thresh_img = stack_thresholds(gray_undistorted_img, undistorted_img, ksize)
#f, (a1, a2, a3, a4, a5, a6) = plt.subplots(6, 1, figsize=(20,30))
#a1.imshow(gradx, cmap='gray')
#a1.set_title("sobel_x")
#a2.imshow(grady, cmap='gray')
#a2.set_title("sobel_y")
#a3.imshow(mag_binary, cmap='gray')
#a3.set_title("gradient magnitude")
#a4.imshow(dir_binary, cmap='gray')
#a4.set_title("gradient direction")
#a5.imshow(color_channel, cmap='gray')
#a5.set_title("S color channel")
#a6.imshow(thresh_img, cmap='gray')
#a6.set_title("Everything")

#Writeup
#thresh_img = stack_thresholds(gray_undistorted_img, undistorted_img, 9)
#cv2.imwrite("writeup_images/binary.jpg", thresh_img*255)

In [37]:
## Apply a perspective transform to rectify binary image ("birds-eye view"). ##

def perspective_transform(thresh_img):
    #Points are hard-coded from image viewer window    
    corners = np.float32([[250, 700], [585, 460], [700, 460], [1060, 700]])
    #Offset from top right and top left
    top_left = np.array([corners[0, 0], 0])
    top_right = np.array([corners[3, 0], 0])
    offset = [50, 0]
    #Get image size
    img_size = (thresh_img.shape[1], thresh_img.shape[0])
    #Define source points and destination points
    src = np.float32([corners[0], corners[1], corners[2], corners[3]])
    dst = np.float32([corners[0] + offset, top_left + offset, top_right - offset, corners[3] - offset])

    #Transform
    M = cv2.getPerspectiveTransform(src, dst)
    pers_trans = cv2.warpPerspective(thresh_img, M, img_size)
    #Inverse matrix to use later
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    return pers_trans, Minv


#TEST
#pers_trans, Minv = perspective_transform(thresh_img)
#plt.imshow(pers_trans, cmap='gray')

#Writeup
#pers_trans, Minv = perspective_transform(thresh_img)
#cv2.imwrite("writeup_images/perspective_transform.jpg", pers_trans*255)

In [38]:
## Detect lane pixels and fit to find the lane boundary. ##

def find_lane_pixels(perspective_transform):
    #Create histogram of bottom half of image
    histogram = np.sum(perspective_transform[perspective_transform.shape[0]//2:,:], axis=0)
    midpoint = np.int(histogram.shape[0]//2)
    #Create an output image to draw on and visualize the result
    out_img = np.dstack((perspective_transform, perspective_transform, perspective_transform))
    #Starting x value of left line
    leftx_base = np.argmax(histogram[:midpoint])
    #Starting x value of right line
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    #Number of sliding windows
    nwindows = 9
    #Width of windows +/- margin
    margin = 100
    #Minimum number of pixels to recenter window
    minpix = 50
    
    #Find height of each windown
    win_height = np.int(perspective_transform.shape[0]/nwindows)
    #x and y positions of all nonzero pixels in the image
    nonzero = perspective_transform.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    #Current x positions initially set to starting values
    leftx_current = leftx_base
    rightx_current = rightx_base
    #Lists will receive lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    #Traverse each window
    for window in range(nwindows):
        #Identify window boundaries
        win_y_low = perspective_transform.shape[0] - (window+1)*win_height
        win_y_high = perspective_transform.shape[0] - window*win_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
    
        #Identify the nonzero pixels in x and y within the window 
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        #Add these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        #If > minpix pixels are found, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    #Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        #Avoids an error if the above is not implemented fully
        pass
    
    #Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(perspective_transform, leftx, lefty, rightx, righty):
    #Fit polynomials to each line
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    #Generate x and y values for plotting
    ploty = np.linspace(0, perspective_transform.shape[0]-1, perspective_transform.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]    

    return ploty, left_fitx, right_fitx


#TEST
#leftx, lefty, rightx, righty, out_img = find_lane_pixels(pers_trans)
#ploty, left_fitx, right_fitx = fit_polynomial(out_img, leftx, lefty, rightx, righty)
#plt.imshow(poly_lines)
#plt.plot(left_fitx, ploty, color='red')
#plt.plot(right_fitx, ploty, color='red')

#Writeup
#leftx, lefty, rightx, righty, poly_lines = find_lane_pixels(pers_trans)
#ploty, left_fitx, right_fitx = fit_polynomial(poly_lines, leftx, lefty, rightx, righty)
#plt.imshow(poly_lines)
#plt.plot(left_fitx, ploty, color='red')
#plt.plot(right_fitx, ploty, color='red')
#plt.savefig("writeup_images/poly_lines.jpg")

In [39]:
## Determine the curvature of the lane and vehicle position with respect to center. ##

def measure_curvature(ploty, left_fitx, right_fitx):
    #Max y value used to determine radius of curvature
    y_eval = np.max(ploty)
    
    #Find polynomials in meters
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    #Calculate radius of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad, left_fit_cr, right_fit_cr


def measure_distance_from_center(poly_lines, left_fit_cr, right_fit_cr):
    #image height and width
    height = poly_lines.shape[0] * ym_per_pix
    width = poly_lines.shape[1] * xm_per_pix
    
    #Finding x intercepts in meters
    left_int = left_fit_cr[0]*height**2 + left_fit_cr[1]*height + left_fit_cr[2]
    right_int = right_fit_cr[0]*height**2 + right_fit_cr[1]*height + right_fit_cr[2]
    #Find lane center
    lane_center = (left_int + right_int) / 2.0
    
    #calculate center of car
    car_center = width / 2.0
    
    #calculate distance from center of car to center of lane
    car_deviation = car_center - lane_center
    return car_deviation


#TEST
#ym_per_pix = 30/720 # meters per pixel in y dimension
#xm_per_pix = 3.7/700 # meters per pixel in x dimension
#left_curverad, right_curverad, left_fit_cr, right_fit_cr = measure_curvature(ploty, left_fitx, right_fitx)
#print(left_curverad, right_curverad, left_fit_cr, right_fit_cr)
#car_deviation = measure_distance_from_center(poly_lines, left_fit_cr, right_fit_cr)
#print(car_deviation)

#Writeup
#ym_per_pix = 30/720 # meters per pixel in y dimension
#xm_per_pix = 3.7/700 # meters per pixel in x dimension
#left_curverad, right_curverad, left_fit_cr, right_fit_cr = measure_curvature(ploty, left_fitx, right_fitx)
#car_deviation = measure_distance_from_center(poly_lines, left_fit_cr, right_fit_cr)

In [40]:
## Warp the detected lane boundaries back onto the original image. ##

def draw_on_orig(pers_trans, raw_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, car_deviation, Minv):
    #Create an image to draw the lines on
    img_zero = np.zeros_like(pers_trans).astype(np.uint8)
    img_with_drawing = np.dstack((img_zero, img_zero, img_zero))

    #Input x and y points into cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    #Draw the lines onto warped blank image
    cv2.fillPoly(img_with_drawing, np.int_([pts]), (0,255, 0))

    #Use inverse matrix to unwarp image
    unwarp = cv2.warpPerspective(img_with_drawing, Minv, (raw_img.shape[1], raw_img.shape[0])) 
    #Add to original image
    final_result = cv2.addWeighted(raw_img, 1, unwarp, 0.3, 0)
    
    #Write text
    curv_rad_text = "Radius of Curvature = " + str(np.round((left_curverad+right_curverad)/2)) + "m"
    car_deviation = round(car_deviation, 2)
    #Determine if car is left of or right of center
    if car_deviation < 0:
        car_deviation = (abs(car_deviation)).astype(str)
        car_deviation_text = "Vehicle is " + car_deviation + "m left of center"
    elif car_deviation > 0:
        car_deviation = (abs(car_deviation)).astype(str)
        car_deviation_text = "Vehicle is " + car_deviation + "m right of center"
    else:
        car_deviation_text = "Vehicle is centered"
    
    cv2.putText(final_result, curv_rad_text, (25,75), cv2.FONT_HERSHEY_DUPLEX , 2, (255,255,255), 2)
    cv2.putText(final_result, car_deviation_text, (25,150), cv2.FONT_HERSHEY_DUPLEX , 2, (255,255,255), 2)
    
    return final_result
    
    
#TEST
#result = draw_on_orig(pers_trans, raw_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, car_deviation, Minv)
#plt.imshow(result)

#Writeup
#result = draw_on_orig(pers_trans, raw_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, car_deviation, Minv)
#r,g,b = cv2.split(result)
#result = cv2.merge((b,g,r))
#cv2.imwrite("writeup_images/result.jpg", result)

In [41]:
## Combine everything. ##

def process_image(raw_img):
    #Undistort image
    undistorted_img = undistort_img(raw_img, mtx, dist)
    #Convert image to gray
    gray_undistorted_img = cv2.cvtColor(undistorted_img, cv2.COLOR_RGB2GRAY)
    #Apply thresholds to image
    thresh_img = stack_thresholds(gray_undistorted_img, undistorted_img, ksize)
    #Perfom perspective transform on image
    pers_trans, Minv = perspective_transform(thresh_img)
    #Find lane pixels
    leftx, lefty, rightx, righty, poly_lines = find_lane_pixels(pers_trans)
    #Fit polynomial to lanes
    ploty, left_fitx, right_fitx = fit_polynomial(poly_lines, leftx, lefty, rightx, righty)
    #Measure curvature of lanes
    left_curverad, right_curverad, left_fit_cr, right_fit_cr = measure_curvature(ploty, left_fitx, right_fitx)
    #Measure car deviation from center of lane
    car_deviation = measure_distance_from_center(poly_lines, left_fit_cr, right_fit_cr)
    #Add graphics and text to original image
    result = draw_on_orig(pers_trans, raw_img, left_fitx, right_fitx, ploty, left_curverad, right_curverad, car_deviation, Minv)
    return result
    

In [42]:
## Generate test images. ##

#Calibrate Camera
ret, mtx, dist, rvecs, tvecs = calibrate_camera("camera_cal/")

#Setting kernel size
ksize = 9
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

#Set input and output folders
input_folder = "test_images/"
output_folder = "output_images/"
raw_files = os.listdir(input_folder)

#Loop through input folder and save to output folder
for file in raw_files:
    raw_img = mpimg.imread(input_folder + file)
    result = process_image(raw_img)
    r,g,b = cv2.split(result)
    result = cv2.merge((b,g,r))
    cv2.imwrite(output_folder + file + ".jpg", result)


In [32]:
## Run pipeline on video. ##

#Calibrate Camera
ret, mtx, dist, rvecs, tvecs = calibrate_camera("camera_cal/")

#Setting kernel size
ksize = 9
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

video_output = "output_video.mp4"
clip1 = VideoFileClip("project_video.mp4")#.subclip(0,5)
clip1_output = clip1.fl_image(process_image)
%time clip1_output.write_videofile(video_output, audio=False)


t:   0%|                                                                            | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Wall time: 5min 38s


In [33]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))